In [13]:
import sys
import json
import csv
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
from collections import Counter
from sklearn.base import TransformerMixin

In [14]:
businessjson = 'dataset/business.json'
checkinjson = 'dataset/checkin.json'

# Load Business data
business_json_lines = [json.loads( l.strip() ) for l in open(businessjson).readlines() ]
df = json_normalize(business_json_lines)



In [ ]:
print df[0]

In [15]:
# Get records from Yelp set for only restaurants
df = df[df['categories'].astype(str).str.lower().str.contains('restaurant') == True]
df = df.reset_index(drop=True)

#Remove Columns of least significance
df = df.drop(['address','attributes.AcceptsInsurance','attributes.AgesAllowed','attributes.BYOB','attributes.ByAppointmentOnly','attributes.Corkage','attributes.DietaryRestrictions.dairy-free','attributes.DietaryRestrictions.gluten-free','attributes.DietaryRestrictions.halal','attributes.DietaryRestrictions.kosher','attributes.DietaryRestrictions.soy-free','attributes.DietaryRestrictions.vegan','attributes.DietaryRestrictions.vegetarian','attributes.HairSpecializesIn.africanamerican','attributes.HairSpecializesIn.asian','attributes.HairSpecializesIn.coloring','attributes.HairSpecializesIn.curly','attributes.HairSpecializesIn.extensions','attributes.HairSpecializesIn.kids','attributes.HairSpecializesIn.perms','attributes.HairSpecializesIn.straightperms','attributes.Open24Hours','attributes.RestaurantsCounterService','business_id','categories','hours.Friday','hours.Monday','hours.Saturday','hours.Sunday','hours.Thursday','hours.Tuesday','hours.Wednesday','latitude','longitude','name','neighborhood','state','city','postal_code'],axis=1)
df_binary = df.copy()
# Classify the records
df_binary['target'] = [1 if df_binary['stars'][x] >= 3.5 and df_binary['review_count'][x] >= 30 else 0 for x in range(len(df_binary))]
df_binary.to_csv('business_csv_filter.csv', encoding='utf-8', index=False)
print df_binary['target'].value_counts()



0    37140
1    14485
Name: target, dtype: int64


In [16]:
headers = list(df.select_dtypes(include=['object']).columns.values)
#print headers
df = pd.get_dummies(data=df, columns=headers)
df = df.fillna(0)


print df.head(10)

   attributes.RestaurantsPriceRange2  is_open  review_count  stars  \
0                                2.0        0             4    4.5   
1                                1.0        0            10    4.5   
2                                2.0        1            21    2.0   
3                                0.0        1             3    3.0   
4                                1.0        1            15    3.0   
5                                2.0        1             6    2.5   
6                                2.0        1            45    3.0   
7                                2.0        1            41    3.0   
8                                2.0        1            51    2.5   
9                                2.0        1             4    4.0   

   attributes.Alcohol_beer_and_wine  attributes.Alcohol_full_bar  \
0                                 0                            0   
1                                 0                            0   
2                        

In [21]:
df_multi_train = df.loc[0:41300]
df_multi_train.to_csv('business_final_train.csv', encoding='utf-8', index=False)

df_multi_test = df.loc[41300:]
df_multi_test.to_csv('business_final_test.csv', encoding='utf-8', index=False)

In [ ]:
# Seggregting data to positive and negative buckets.
# Splitting train and test data for binary.
pos_bucket = df_binary.loc[df_binary['target'] == 1].reset_index(drop=True)
neg_bucket = df_binary.loc[df_binary['target'] == 0].reset_index(drop=True)

df_pos_train = df_binary.loc[0:11588]
df_pos_test = df_binary.loc[11588:]
df_neg_train = df_binary.loc[0:11884]
df_neg_test = df_binary.loc[11884:]

print df_pos_train.shape[0],"df_pos_train"
print df_pos_test.shape[0],"df_pos_test"
print df_neg_train.shape[0],"df_neg_train"
print df_neg_test.shape[0],"df_neg_test"


df_train = pd.concat([df_pos_train, df_neg_train], axis=0)
df_train = df_train.reset_index(drop=True)

print df_train.shape[0],"df_train"

df_test = pd.concat([df_pos_test, df_neg_test], axis=0)
df_test = df_test.reset_index(drop=True)

print df_test.shape[0],"df_test"
print df_train.loc[0:1]," df_train.loc_ahead"
print df_train.loc[12589]," df_train.loc_behind"

In [5]:
def build_matrix(lists):
    """ 
            Build sparse matrix from a list of documents, 
            each of which is a list of word/terms in the document.  
    """


    docs = []
    for list in lists:
        list = list.strip()
        docs.append([int(n) if n != '' else 0 for n in list.split(' ')])
    nrows = len(docs)
    idx = {}
    tid = 0
    nnz = 0
    for d in docs:
        nnz += len(set(d))
        for w in d:
            if w not in idx:
                idx[w] = tid
                tid += 1
    ncols = len(idx)


    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = [k for k,_ in cnt.most_common()]
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat,idx


In [6]:
dataset_csr = build_matrix(df)

ValueError: invalid literal for int() with base 10: 'attributes.RestaurantsPriceRange2'

In [ ]:
from sklearn import preprocessing
train_norm = preprocessing.normalize(dataset_csr, norm='l2')

In [ ]:
# for filling the missing values
from sklearn.base import TransformerMixin

class SeriesImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        If the Series is of dtype Object, then impute with the most frequent object.
        If the Series is not of dtype Object, then impute with the mean.  

        """
    def fit(self, X, y=None):
        if   X.dtype == np.dtype('O'): self.fill = X.value_counts().index[0]
        else                            : self.fill = X.mean()
        return self

    def transform(self, X, y=None):
       return X.fillna(self.fill)


In [ ]:
print Counter(df['attributes.Alcohol'])
a  = SeriesImputer()   # Initialize the imputer
a.fit(df['attributes.Alcohol'])              # Fit the imputer
df['attributes.Alcohol'] = a.transform(df['attributes.Alcohol'])
print Counter(df['attributes.Alcohol'])

In [ ]:
print Counter(df['attributes.Ambience.casual'])
print df.groupby(['attributes.Ambience.casual','target']).size()


In [20]:
# Seggregting data to positive and negative buckets.
#print df.loc[0]
pos_bucket = df_binary.loc[df_binary['target'] == 1].reset_index(drop=True)
neg_bucket = df_binary.loc[df_binary['target'] == 0].reset_index(drop=True)

df_pos_train = df_binary.loc[0:11588]
df_pos_test = df_binary.loc[11588:]
df_neg_train = df_binary.loc[0:11884]
df_neg_test = df_binary.loc[11884:]

print df_pos_train.shape[0],"df_pos_train"
print df_pos_test.shape[0],"df_pos_test"
print df_neg_train.shape[0],"df_neg_train"
print df_neg_test.shape[0],"df_neg_test"


df_train = pd.concat([df_pos_train, df_neg_train], axis=0)
df_train = df_train.reset_index(drop=True)

print df_train.shape[0],"df_train"

df_test = pd.concat([df_pos_test, df_neg_test], axis=0)
df_test = df_test.reset_index(drop=True)

print df_test.shape[0],"df_test"
print df_train.loc[0:1]," df_train.loc_ahead"
print df_train.loc[12589]," df_train.loc_behind"

11589 df_pos_train
40037 df_pos_test
11885 df_neg_train
39741 df_neg_test
23474 df_train
79778 df_test
  attributes.Alcohol attributes.Ambience.casual attributes.Ambience.classy  \
0                NaN                      False                      False   
1               none                      False                      False   

  attributes.Ambience.divey attributes.Ambience.hipster  \
0                     False                       False   
1                     False                       False   

  attributes.Ambience.intimate attributes.Ambience.romantic  \
0                        False                        False   
1                        False                        False   

  attributes.Ambience.touristy attributes.Ambience.trendy  \
0                        False                      False   
1                        False                      False   

  attributes.Ambience.upscale  ...   attributes.RestaurantsReservations  \
0                       False  ... 

In [ ]:
print Counter(pos_bucket['attributes.Alcohol'])
print pos_bucket.groupby(['attributes.Alcohol','target']).size()

pos_bucket.loc[pos_bucket['attributes.Alcohol'].isnull(),'attributes.Alcohol'] = 'full_bar'

print len(pos_bucket.loc[pos_bucket['attributes.Alcohol'].isnull(),'attributes.Alcohol'])

print pos_bucket['attributes.Alcohol']
print Counter(pos_bucket['attributes.Alcohol']),"$"
print pos_bucket.groupby(['attributes.Alcohol','target']).size(),"^"
